In [ ]:
import torch
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
def rescale(vec):
    min_val = vec.min()
    max_val = vec.max()
    if min_val == max_val:
        return torch.zeros_like(vec)
    else:
        return (vec - min_val) / (max_val - min_val)

In [ ]:
def sparse_vector_field(coords, vecs, x_dim=100, y_dim=100):
    x, y = coords.t()
    x = rescale(x) * (x_dim - 1)
    y = rescale(y) * (y_dim - 1)

    indices = torch.stack([x.round().long(), y.round().long()]).t()
    _, count_indices, counts = np.unique(indices, axis=0, return_inverse=True, return_counts=True)
    counts = torch.tensor(counts[count_indices]).unsqueeze(1)

    return torch.sparse.FloatTensor(
        indices.t(),
        vecs / counts,
        torch.Size([x_dim, y_dim, 2])
    ).coalesce()

In [ ]:
def sparse_vector_field_2(coords, vecs, x_dim=100, y_dim=100):
    x, y = coords.t()
    x = rescale(x) * (x_dim - 1)
    y = rescale(y) * (y_dim - 1)

    indices = torch.stack([x.round().long(), y.round().long()]).t()
    _, count_indices, counts = np.unique(indices, axis=0, return_inverse=True, return_counts=True)
    counts = torch.tensor(counts[count_indices]).unsqueeze(1)

    field = torch.zeros((x_dim, y_dim, 2))
    for idx, ct, v in zip(indices, counts, vecs):
        field[tuple(idx)] += v / ct

    return field

In [ ]:
def divergence(coords, vec, x_dim=100,y_dim=100):
    vf = sparse_vector_field_2(coords, vec, x_dim=x_dim, y_dim=y_dim)#.to_dense()

    mask = vf.eq(torch.zeros_like(vf))
    mask = mask[:,:,0] * mask[:,:,1]
    mask = mask[1:,1:] + mask[:-1,:-1]
    mask = (1 - mask.float())

    x1 = vf[:-1,:,0]
    x2 = vf[1:,:,0]
    x_div = x2 - x1
    x_div_sub = x_div[:,:-1] + x_div[:,1:]
    if x_div_sub.eq(0.).all():
        raise Exception('Not enough samples in x-direction')

    y1 = vf[:,:-1,1]
    y2 = vf[:,1:,1]
    y_div = y2 - y1
    y_div_sub = y_div[0:-1,:] + y_div[1:,:]
    if y_div_sub.eq(0.).all():
        raise Exception('Not enough samples in y-direction')

    div = (x_div_sub + y_div_sub) / 2.
    
    return mask * div

# Tests

## Fixed example

In [ ]:
coords = torch.stack((torch.tensor([0,0,0,1,1,1,2,2,2,3,3,3]), torch.tensor([0,1,2,0,1,2,0,1,2,0,1,2]))).t().float() 
vecs = torch.stack((torch.tensor([0,1,2,-1,0,1,-4,-3,-2,-9,-8,-7]), torch.tensor([1,10,100,0,9,99,-1,8,98,-2,7,97]))).t().float()
#field = sparse_vector_field_2(coords, vecs, x_dim=4, y_dim=4)
field = sparse_vector_field_2(coords, vecs, x_dim=6, y_dim=3)
field

## F(x,y) = (x, y)

In [ ]:
coords = torch.stack(torch.meshgrid([torch.arange(-50,51), torch.arange(-50,51)])).transpose(0,2).reshape(-1,2).float()

In [ ]:
#rand_idx = np.unique(np.random.randint(0, high=len(coords), size=2000))

In [ ]:
divergence(coords, coords, x_dim=101, y_dim=101)

In [ ]:
#v = torch.rand(10000,2)
#c = v.clone()
divs = []
for i in [10,20,30,40,50,60,70,80,90,100,150,200]:
    divs.append(divergence(coords,coords, x_dim=i, y_dim=i).sum() / i**2)

In [ ]:
plt.plot(divs)

## Sparseness of gradient

In [ ]:
v = torch.rand(500,2, requires_grad=True)
c = v.clone().detach()
field = sparse_vector_field_2(c, 10*v, x_dim=10,y_dim=10)
div = divergence(c, 10*v, x_dim=10,y_dim=10)
div.sum().backward()
grad = v.grad
grad

In [ ]:
(grad[:,0].bool() + grad[:,1].bool()).sum().float() / grad.shape[0]

In [ ]:
plt.quiver(c[:,0], c[:,1], v.detach()[:,0], v.detach()[:,1])

In [ ]:
grad_pts = c[(grad[:,0].bool() + grad[:,1].bool())]
plt.scatter(grad_pts[:,0], grad_pts[:,1])

## Real data

In [ ]:
coords = np.loadtxt('projected_output_test.csv').reshape(15, 60000, 2)

In [ ]:
coords_ep1 = torch.tensor(coords[1], requires_grad=True)
c = torch.tensor(coords[0])
v = coords_ep1 - c

In [ ]:
plt.quiver(c[::100,0],c[::100,1],v.detach()[::100,0],v.detach()[::100,1])

In [ ]:
div = divergence(c, v, x_dim=50, y_dim=50)

In [ ]:
plt.imshow(reversed(div.detach().t()))

In [ ]:
div.sum().backward()

In [ ]:
coord_grad = coords_ep1.grad

In [ ]:
coord_grad_pts = coords_ep1.detach()[(coord_grad[:,0].bool() + coord_grad[:,1].bool())]
plt.scatter(coord_grad_pts[:,0], coord_grad_pts[:,1])

In [ ]:
coords_ep1 = torch.tensor(coords[1], requires_grad=True)
c = torch.tensor(coords[0])
v = coords_ep1 - c
div = divergence(c, v, x_dim=50, y_dim=50)
div.pow(3).sum().backward()

In [ ]:
coord_grad = coords_ep1.grad
coord_grad_pts = coords_ep1.detach()[(coord_grad[:,0].bool() + coord_grad[:,1].bool())]
plt.scatter(coord_grad_pts[:,0], coord_grad_pts[:,1])

In [ ]:
#TODO: write test vector fields and check if divergence is correct
#TODO: integrating divergence yields only boundary terms -> introduce non-linearity?
#TODO: 